In [ ]:
from pyits.__main__ import Arguments
import argparse
import dataclasses
from dataclasses import dataclass, fields, MISSING, field
import logging

import gsd
import pandas as pd
import numpy as np
import jax
import jax.numpy as jnp
from jax import Array
from jax.typing import ArrayLike

# HDTV

In [ ]:
args = Arguments(inp='/Users/krzysiek/Documents/gsd-bayes/data/subjective_quality_datasets.csv', out='log/hdtv.csv')

In [ ]:
df = pd.read_csv(args.inp)

infos = []

#hdtv
for eid in range(1,7):
    dfe = df.loc[df.Experiment == eid, :]

    pvss = dfe['PVS_id'].unique()

    for pvs in pvss:
        arr = dfe.loc[dfe.PVS_id == pvs, 'Score'].to_numpy()
        counts_arr = gsd.sufficient_statistic(arr)

        hat, opt_state = gsd.fit_mle(counts_arr)

        fhat = jax.tree_util.tree_map(float, hat)
        info = dict(eid=eid, pvs=pvs, **fhat._asdict())
        print(info)
        infos.append(info)

pd.DataFrame(infos).to_csv(args.out)



# Its

data for GSD analysis. week is video, q2 is subject answer. Generated from phase4_current_results_p1204.csv

In [ ]:
args = Arguments(inp='../4phase_real_analysis/for_krzysiek.csv', out='log/its.csv')

In [ ]:
df = pd.read_csv(args.inp)
df7=df.loc[df.n==7,:]

In [ ]:
infos=[]
ws = df7.week.unique()
for w in ws:
    scores = df7.loc[df7.week==w,'q2'].to_numpy()
    counts_arr =  gsd.sufficient_statistic(scores)
    hat, opt_state = gsd.fit_mle(counts_arr)

    fhat = jax.tree_util.tree_map(float, hat)
    info = dict(pvs=w, **fhat._asdict())
    print(info)
    infos.append(info)

In [ ]:
pd.DataFrame(infos).to_csv(args.out)

# Figure

In [ ]:
import seaborn as sns

hdtv = pd.read_csv('log/hdtv.csv')
its = pd.read_csv('log/its.csv')

In [ ]:
import matplotlib.pyplot as plt

sns.displot(data=hdtv, x='psi',y='rho', kind='kde')
sns.scatterplot(data=its,x='psi',y='rho', color='k')
plt.legend(['its'])
plt.title("density of GSD parameters")
plt.savefig('log/psirho_hist2d.pdf')